<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [2]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install 'keras<3.0.0' mediapipe-model-maker

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_models_official-2.16.0-py2.py3-none-any.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting un

Import the required packages.

In [4]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


### Prepare the dataset

This dataset contains about 130 images of 3 different dogs: a Golden Retriever, a Labrador Retriever mix, and a brindle-colored Formosan Mountain Dog. This is an example image from the dataset.

![aci.jpg](https://storage.googleapis.com/tflite-task-tfjs/aci.jpeg)

We start with downloading the dataset.

In [5]:
!wget https://storage.googleapis.com/mediapipe-assets/dogs2.zip --no-check-certificate
!unzip dogs2.zip
train_dataset_path = "dogs/train"
validation_dataset_path = "dogs/validate"

--2024-03-25 07:33:48--  https://storage.googleapis.com/mediapipe-assets/dogs2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.207, 142.250.4.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58926065 (56M) [application/zip]
Saving to: ‘dogs2.zip’

dogs2.zip           100%[===================>]  56.20M  17.8MB/s    in 3.5s    

2024-03-25 07:33:52 (16.0 MB/s) - ‘dogs2.zip’ saved [58926065/58926065]

Archive:  dogs2.zip
   creating: dogs copy/
   creating: dogs copy/validate/
  inflating: dogs copy/.DS_Store     
  inflating: __MACOSX/dogs copy/._.DS_Store  
   creating: dogs copy/train/
  inflating: dogs copy/validate/.DS_Store  
  inflating: __MACOSX/dogs copy/validate/._.DS_Store  
   creating: dogs copy/validate/images/
   creating: dogs copy/validate/Annotations/
  inflating: dogs copy/train/.DS_Store  
  inflating: __MACOSX/do

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [6]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    'dogs copy/train',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    'dogs copy/validate',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [7]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=50, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


13/13 [==============================] - 70s 1s/step - total_loss: 6.8362 - cls_loss: 6.5766 - box_loss: 0.0041 - model_loss: 6.7813 - val_total_loss: 1.3038 - val_cls_loss: 1.1607 - val_box_loss: 0.0018 - val_model_loss: 1.2489
Epoch 2/50
13/13 [==============================] - 4s 296ms/step - total_loss: 1.4056 - cls_loss: 1.1882 - box_loss: 0.0032 - model_loss: 1.3506 - val_total_loss: 1.2544 - val_cls_loss: 1.1101 - val_box_loss: 0.0018 - val_model_loss: 1.1994
Epoch 3/50
13/13 [==============================] - 5s 369ms/step - total_loss: 1.2601 - cls_loss: 1.1072 - box_loss: 0.0020 - model_loss: 1.2051 - val_total_loss: 1.1717 - val_cls_loss: 1.0268 - val_box_loss: 0.0018 - val_model_loss: 1.1168
Epoch 4/50
13/13 [==============================] - 4s 300ms/step - total_loss: 1.1069 - cls_loss: 0.9844 - box_loss: 0.0013 - model_loss: 1.0519 - val_total_loss: 1.0633 - val_cls_loss: 0.9164 - val_box_loss: 0.0018 - val_model_loss: 1.0084
Epoch 5/50
13/13 [===========================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [8]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

7/7 [==============================] - 2s 71ms/step - total_loss: 0.6390 - cls_loss: 0.4931 - box_loss: 0.0018 - model_loss: 0.5841
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.512
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.690
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.633
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.531
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.632
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.755
 Average Recall     (AR) @[ Io

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [9]:
model.export_model('dogs.tflite')
!ls exported_model
files.download('exported_model/dogs.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


dogs.tflite  float_ckpt.data-00000-of-00001  float_ckpt.index  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>